**Notebook preparation and importing libraries**

In [1]:
from pathlib import Path
from hvac import Quantity
from hvac.fluids import Fluid, HumidAir
from hvac.vapor_compression import VariableSpeedCompressor
from hvac.vapor_compression.machine_bis import SingleStageVaporCompressionMachine
from hvac.heat_transfer.heat_exchanger.fin_tube.air_evaporator import PlainFinTubeCounterFlowEvaporator
from hvac.heat_transfer.heat_exchanger.fin_tube.air_condenser import PlainFinTubeCounterFlowCondenser

In [2]:
from hvac.logging import ModuleLogger
logger = ModuleLogger.get_logger('hvac.fluids.fluid')
logger.setLevel(ModuleLogger.ERROR)

In [3]:
Q_ = Quantity

# Rating of a Single-Stage Vapor Compression Machine

A single-stage vapor compression machine is composed of a compressor, an evaporator, a condenser, and an expansion device. Air at a given state and mass flow rate enters the evaporator. Refrigerant in the evaporator absorbs heat from the air stream. At the condenser, air also enters at a given state and mass flow rate. Refrigerant in the condenser rejects heat to this air stream. The expansion device regulates the mass flow rate of refrigerant in order to maintain the set degree of superheat. The compressor (either with fixed or variable speed drive) runs at a given compressor speed. The question is what operating point the vapor compression machine will have under the given operating conditions.

## Configuration of the Vapor Compression Machine

To configure a single-stage vapor compression machine, we first need to define compressor, evaporator and condenser.

**Compressor**<br>
Based on preliminary calculations (see the first notebook of this series), a variable speed compressor with refrigerant R134a has been selected using the selection software of a manufacturer. The polynomial coefficients of the functions describing the compressor performance quantities were saved in a csv-file.   

Define the refrigerant:

In [4]:
R134a = Fluid('R134a')

In [5]:
compressor = VariableSpeedCompressor(
    coeff_file=Path("compressor_data/VTZ038-G_R134a.csv"),
    refrigerant_type=R134a,
    units={'m_dot': 'kg / hr', 'speed': '1 / s'}
)

**Evaporator**<br>
We will use the same air evaporator defined in notebook no. 2 (plain fin-tube counterflow evaporator).

In [6]:
evaporator = PlainFinTubeCounterFlowEvaporator(
    L1=Q_(0.844, 'm'),
    L3=Q_(0.211, 'm'),
    N_r=6,
    S_t=Q_(25.4, 'mm'),
    S_l=Q_(22.0, 'mm'),
    D_i=Q_(8.422, 'mm'),
    D_o=Q_(10.2, 'mm'),
    t_f=Q_(0.3302, 'mm'),
    N_f=1 / Q_(3.175, 'mm')
)

**Condenser**<br>
We will use the same air condenser defined in notebook no. 3 (plain fin-tube counterflow condenser).

In [7]:
condenser = PlainFinTubeCounterFlowCondenser(
    L1=Q_(0.550, 'm'),
    L3=Q_(0.275, 'm'),
    N_r=10,
    S_t=Q_(25.4, 'mm'),
    S_l=Q_(22.0, 'mm'),
    D_i=Q_(8.422, 'mm'),
    D_o=Q_(10.2, 'mm'),
    t_f=Q_(0.3302, 'mm'),
    N_f=1 / Q_(3.175, 'mm')
)

**Vapor compression machine**

In [8]:
machine = SingleStageVaporCompressionMachine(evaporator, condenser, compressor, R134a)

## Specify Known Operating Conditions

- mass flow rate of air through the evaporator
- mass flow rate of air through the condenser
- air state at evaporator inlet
- air state at condenser inlet
- degree of superheat set on the expansion device
- compressor speed (only in case of a variable speed compressor)

In [9]:
machine.set_operating_conditions(
    evp_m_dot_air=Q_(1500, 'kg / hr'),
    cnd_m_dot_air=Q_(2454.820, 'kg / hr'),
    evp_air_in=HumidAir(Tdb=Q_(24, 'degC'), RH=Q_(50, 'pct')),
    cnd_air_in=HumidAir(Tdb=Q_(35.0, 'degC'), RH=Q_(30, 'pct')),
    dT_super=Q_(10, 'K'),
    n_cmp=Q_(4000, '1 / min')
)

## Steady-State Operation

To determine the steady-state operating parameters of the machine an iterative solution technique is used as set out in Chapter 14 of *Stoecker, W. F., & Jones, J. W. (1982). Refrigeration and air conditioning. McGraw-Hill International Editions, Mechanical Technology Series.*

In [11]:
%%time 
machine.simulate()

[hvac.vapor_compression.machine_bis | DEBUG] iteration 1: try T_evp = 4.000 °C; T_cnd = 55.000 °C
[hvac.vapor_compression.machine_bis | DEBUG] iteration 2: try T_evp = 13.154 °C; T_cnd = 53.043 °C
[hvac.vapor_compression.machine_bis | DEBUG] iteration 3: try T_evp = 9.644 °C; T_cnd = 51.815 °C
[hvac.vapor_compression.machine_bis | DEBUG] iteration 4: try T_evp = 11.269 °C; T_cnd = 50.707 °C
[hvac.vapor_compression.machine_bis | DEBUG] iteration 5: try T_evp = 10.369 °C; T_cnd = 50.007 °C
[hvac.vapor_compression.machine_bis | DEBUG] iteration 6: try T_evp = 10.755 °C; T_cnd = 49.425 °C
[hvac.vapor_compression.machine_bis | DEBUG] iteration 7: try T_evp = 10.498 °C; T_cnd = 49.097 °C
[hvac.vapor_compression.machine_bis | DEBUG] iteration 8: try T_evp = 10.589 °C; T_cnd = 48.843 °C
[hvac.vapor_compression.machine_bis | DEBUG] iteration 9: try T_evp = 10.514 °C; T_cnd = 48.692 °C
[hvac.vapor_compression.machine_bis | DEBUG] iteration 10: try T_evp = 10.534 °C; T_cnd = 48.584 °C


CPU times: total: 49.2 s
Wall time: 50.2 s


The operating point of the vapor compression machine is defined by the evaporating and condensing temperature:

In [12]:
print(
    f"evaporation temperature = {machine.Te.to('degC'):~P.2f}\n"
    f"condensing temperature = {machine.Tc.to('degC'):~P.2f}"
)

evaporation temperature = 10.51 °C
condensing temperature = 48.52 °C


Other operating characteristics are:

In [13]:
print(
    f"heat absorption rate = {machine.Qc_dot.to('kW'):~P.3f}\n"
    f"heat rejection rate = {machine.Qh_dot.to('kW'):~P.3f}\n"
    f"compressor power = {machine.Wc_dot.to('kW'):~P.3f}\n"
    f"COP = {machine.COP.to('frac'):~P.2f}\n"
    f"refrigerant mass flow rate = {machine.m_dot.to('kg / hr'):~P.3f}"
)

heat absorption rate = 5.698 kW
heat rejection rate = 7.473 kW
compressor power = 1.774 kW
COP = 3.21 frac
refrigerant mass flow rate = 135.213 kg/h


The state of air at the outlet of evaporator and condenser:

In [14]:
print(
    f"evaporator air out = {evaporator.air_out.Tdb.to('degC'):~P.2f} DB, "
    f"{evaporator.air_out.RH.to('pct'):~P.1f} RH\n"
    f"condenser air out = {condenser.air_out.Tdb.to('degC'):~P.2f} DB, "
    f"{condenser.air_out.RH.to('pct'):~P.1f} RH"
)

evaporator air out = 13.23 °C DB, 87.4 % RH
condenser air out = 45.80 °C DB, 16.9 % RH


The different states of refrigerant along the vapor compression cycle:

In [15]:
print(
    f"suction gas temperature = {machine.suction_gas.T.to('degC'):~P.2f}\n"
    f"discharge gas temperature = {machine.discharge_gas.T.to('degC'):~P.2f}\n"
    f"liquid temperature = {machine.liquid.T.to('degC'):~P.2f}\n"
    f"liquid/vapor mixture temperature = {machine.mixture.T.to('degC'):~P.2f}\n"
    f"subcooling degree = {machine.sub_cooling.to('K'):~P.2f}"
)

suction gas temperature = 20.51 °C
discharge gas temperature = 82.52 °C
liquid temperature = 43.37 °C
liquid/vapor mixture temperature = 10.51 °C
subcooling degree = 5.15 K


Air-side pressure drop across evaporator and condenser:

In [16]:
print(
    f"evaporator air-side pressure drop = {evaporator.dP_air.to('Pa'):~P.1f}\n"
    f"condenser air-side pressure drop = {condenser.dP_air.to('Pa'):~P.1f}"
)

evaporator air-side pressure drop = 45.6 Pa
condenser air-side pressure drop = 268.7 Pa
